## 네이버 금융 개별종목 수집
* FinanceDataReader를 통해 수집했던 데이터를 네이버 증권 웹 페이지를 통해 직접 수집합니다.


### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 2020년 주요 상장종목
    * 빅히트 : https://finance.naver.com/item/main.nhn?code=352820
    * 카카오게임즈 : https://finance.naver.com/item/main.nhn?code=293490
    * SK바이오팜 : https://finance.naver.com/item/main.nhn?code=326030

## 라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd

## 수집할 URL 정하기

In [2]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "352820"
# item_name = "빅히트"

item_code = "326030"
item_name = "SK바이오팜"
page_no = 1

# 종목 URL 만들기
url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
url

'https://finance.naver.com/item/sise_day.nhn?code=326030&page=1'

## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [3]:
import requests

headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

response = requests.get(url, headers=headers)
# response.text

## BeautifulSoup 을 통한 table 태그 찾기

In [4]:
from bs4 import BeautifulSoup as bs

html = bs(response.text, "lxml")
table = html.select("table")
len(table)

2

In [5]:
type(table)

list

## pandas 코드 한 줄로 데이터 수집하기

In [6]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.
table = pd.read_html(str(table))

In [7]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[1]

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,2,3,4,5,6,7,8,9,10,다음,맨뒤


In [8]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
temp = table[0].dropna()
temp

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.01.29,145000.0,3000.0,149000.0,149500.0,144500.0,254107.0
2,2021.01.28,148000.0,1500.0,148000.0,150500.0,147000.0,200418.0
3,2021.01.27,149500.0,2500.0,152000.0,153000.0,149500.0,167932.0
4,2021.01.26,152000.0,0.0,153000.0,155000.0,151500.0,245969.0
5,2021.01.25,152000.0,1500.0,151000.0,153500.0,150500.0,250843.0
9,2021.01.22,150500.0,1000.0,152500.0,153000.0,149000.0,211286.0
10,2021.01.21,151500.0,3500.0,148000.0,156500.0,147500.0,641065.0
11,2021.01.20,148000.0,0.0,147500.0,148000.0,146000.0,243971.0
12,2021.01.19,148000.0,0.0,146500.0,148000.0,145000.0,274255.0
13,2021.01.18,148000.0,5000.0,151500.0,151500.0,147000.0,419631.0


## 페이지별 데이터 수집 함수 만들기

In [9]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """ 
    url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
    
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

    response = requests.get(url, headers=headers)
    html = bs(response.text, "lxml")
    table = html.select("table")
    table = pd.read_html(str(table))
    temp = table[0].dropna()
    return temp

In [10]:
# 함수가 잘 만들어졌는지 확인하기
get_day_list(item_code, 1)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.01.29,145000.0,3000.0,149000.0,149500.0,144500.0,254107.0
2,2021.01.28,148000.0,1500.0,148000.0,150500.0,147000.0,200418.0
3,2021.01.27,149500.0,2500.0,152000.0,153000.0,149500.0,167932.0
4,2021.01.26,152000.0,0.0,153000.0,155000.0,151500.0,245969.0
5,2021.01.25,152000.0,1500.0,151000.0,153500.0,150500.0,250843.0
9,2021.01.22,150500.0,1000.0,152500.0,153000.0,149000.0,211286.0
10,2021.01.21,151500.0,3500.0,148000.0,156500.0,147500.0,641065.0
11,2021.01.20,148000.0,0.0,147500.0,148000.0,146000.0,243971.0
12,2021.01.19,148000.0,0.0,146500.0,148000.0,145000.0,274255.0
13,2021.01.18,148000.0,5000.0,151500.0,151500.0,147000.0,419631.0


## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [11]:
import time
# web page 시작번호
page_no = 1
# 데이터를 저장할 빈 변수 선언
item_list = []

while True:
    print(page_no)
    temp = get_day_list(item_code, page_no)
    item_list.append(temp)
    
    page_no = page_no + 1
    time.sleep(0.1)
    
    #if temp.shape[0] < 10:
    if page_no > 10:
        break;

1
2
3
4
5
6
7
8
9
10


In [12]:
len(item_list)

10

## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [13]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
df = pd.concat(item_list)

<img src="https://pandas.pydata.org/docs/_images/08_concat_row1.svg">

In [14]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.01.29,145000.0,3000.0,149000.0,149500.0,144500.0,254107.0
2,2021.01.28,148000.0,1500.0,148000.0,150500.0,147000.0,200418.0
3,2021.01.27,149500.0,2500.0,152000.0,153000.0,149500.0,167932.0
4,2021.01.26,152000.0,0.0,153000.0,155000.0,151500.0,245969.0
5,2021.01.25,152000.0,1500.0,151000.0,153500.0,150500.0,250843.0


In [15]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
9,2020.09.09,191000.0,3000.0,184500.0,194000.0,181000.0,752071.0
10,2020.09.08,188000.0,3500.0,193000.0,197000.0,187000.0,688213.0
11,2020.09.07,191500.0,1500.0,193000.0,193500.0,187000.0,588499.0
12,2020.09.04,190000.0,8000.0,177000.0,191500.0,177000.0,1263233.0
13,2020.09.03,182000.0,500.0,183500.0,186500.0,179000.0,564777.0


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [16]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
df["종목코드"] = item_code
df["종목명"] = item_name
df

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
1,2021.01.29,145000.0,3000.0,149000.0,149500.0,144500.0,254107.0,326030,SK바이오팜
2,2021.01.28,148000.0,1500.0,148000.0,150500.0,147000.0,200418.0,326030,SK바이오팜
3,2021.01.27,149500.0,2500.0,152000.0,153000.0,149500.0,167932.0,326030,SK바이오팜
4,2021.01.26,152000.0,0.0,153000.0,155000.0,151500.0,245969.0,326030,SK바이오팜
5,2021.01.25,152000.0,1500.0,151000.0,153500.0,150500.0,250843.0,326030,SK바이오팜
...,...,...,...,...,...,...,...,...,...
9,2020.09.09,191000.0,3000.0,184500.0,194000.0,181000.0,752071.0,326030,SK바이오팜
10,2020.09.08,188000.0,3500.0,193000.0,197000.0,187000.0,688213.0,326030,SK바이오팜
11,2020.09.07,191500.0,1500.0,193000.0,193500.0,187000.0,588499.0,326030,SK바이오팜
12,2020.09.04,190000.0,8000.0,177000.0,191500.0,177000.0,1263233.0,326030,SK바이오팜


## 컬럼 순서 변경하기

In [17]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.
df.columns

Index(['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '종목코드', '종목명'], dtype='object')

In [18]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]

## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [19]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인
df = df.drop_duplicates()
df.shape

(100, 9)

## 기술통계값 구하기

In [20]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,100.00000,100.000000,100.000000,100.000000,100.000000,1.000000e+02
mean,164690.00000,2755.000000,165105.000000,167325.000000,162040.000000,4.330877e+05
std,11958.00311,2878.424618,12100.858764,12312.918198,11450.746524,4.701523e+05
min,140500.00000,0.000000,141000.000000,142500.000000,138000.000000,1.074820e+05
25%,155000.00000,1000.000000,155375.000000,157375.000000,152500.000000,1.934870e+05
50%,166750.00000,2000.000000,167750.000000,169000.000000,162500.000000,2.538250e+05
75%,171750.00000,3500.000000,172625.000000,174125.000000,170500.000000,4.583068e+05
max,191500.00000,16000.000000,194000.000000,197000.000000,187000.000000,3.338448e+06


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [21]:
# 거래량의 과학적 기수법 읽기
1.000000e+02

100.0

In [22]:
4.701523e+05

470152.3

In [23]:
4.701523 * (10 ** 5)

470152.3

## 최근 날짜 구해서 파일명 만들기

In [24]:
# 날짜 column의 첫 row값 확인
date = df.iloc[0]["날짜"]
date

'2021.01.29'

In [25]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
file_name = f"{item_name}_{item_code}_{date}.csv"
file_name

'SK바이오팜_326030_2021.01.29.csv'

## 파일로 저장하기

In [26]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.
df.to_csv(file_name, index=False)

In [27]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다
pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,326030,SK바이오팜,2021.01.29,145000.0,3000.0,149000.0,149500.0,144500.0,254107.0
1,326030,SK바이오팜,2021.01.28,148000.0,1500.0,148000.0,150500.0,147000.0,200418.0
2,326030,SK바이오팜,2021.01.27,149500.0,2500.0,152000.0,153000.0,149500.0,167932.0
3,326030,SK바이오팜,2021.01.26,152000.0,0.0,153000.0,155000.0,151500.0,245969.0
4,326030,SK바이오팜,2021.01.25,152000.0,1500.0,151000.0,153500.0,150500.0,250843.0
...,...,...,...,...,...,...,...,...,...
95,326030,SK바이오팜,2020.09.09,191000.0,3000.0,184500.0,194000.0,181000.0,752071.0
96,326030,SK바이오팜,2020.09.08,188000.0,3500.0,193000.0,197000.0,187000.0,688213.0
97,326030,SK바이오팜,2020.09.07,191500.0,1500.0,193000.0,193500.0,187000.0,588499.0
98,326030,SK바이오팜,2020.09.04,190000.0,8000.0,177000.0,191500.0,177000.0,1263233.0


## 전체 과정을 하나의 함수로 만들기

In [28]:
def get_item_list(item_code, item_name):
    """
    일별 시세를 수집하는 함수
    """
    # web page 시작번호
    page_no = 1
    # 데이터를 저장할 빈 변수 선언
    item_list = []

    while True:
        temp = get_day_list(item_code, page_no)
        item_list.append(temp)

        page_no = page_no + 1
        time.sleep(0.1)

        #if temp.shape[0] < 10:
        if page_no > 10:
            break
            
    df = pd.concat(item_list)
    df["종목코드"] = item_code
    df["종목명"] = item_name
    cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
    df = df[cols]

    return df

In [29]:
item_code = "352820"
item_name = "빅히트"

# item_code = "326030"
# item_name = "SK바이오팜"

get_item_list(item_code, item_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,352820,빅히트,2021.01.29,204500.0,15500.0,222000.0,222000.0,200500.0,765211.0
2,352820,빅히트,2021.01.28,220000.0,12500.0,213000.0,225000.0,208500.0,1653828.0
3,352820,빅히트,2021.01.27,207500.0,3000.0,208000.0,221500.0,206500.0,1003201.0
4,352820,빅히트,2021.01.26,210500.0,7000.0,203500.0,219000.0,194500.0,906368.0
5,352820,빅히트,2021.01.25,203500.0,11000.0,202000.0,217000.0,197500.0,1884652.0
...,...,...,...,...,...,...,...,...,...
4,352820,빅히트,2020.10.15,258000.0,12000.0,270000.0,351000.0,253500.0,6557212.0
1,352820,빅히트,2020.10.20,182500.0,6500.0,190000.0,193500.0,176000.0,1811616.0
2,352820,빅히트,2020.10.19,189000.0,11500.0,198500.0,202500.0,187000.0,1977996.0
3,352820,빅히트,2020.10.16,200500.0,57500.0,253000.0,253000.0,199000.0,4504395.0


## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [30]:
# 도움말 보기
# get_item_list?


In [31]:
# 소스코드 보기
# get_item_list??
